# Autoencoder variacional
## José Díaz Parra y Pablo Sierra Sanz
### Introducción

En este notebook se recoge el código necesario para ejecutar el entrenamiento de un autoencoder dedicado a generar imágenes.
Para la generación de imágenes hemos utilizado un dataset de perros y gatos disponible en el siguiente enlace:

https://www.microsoft.com/en-us/download/details.aspx?id=54765

El dataset viene con un formato dividido en 2 carpetas, una de fotografías de perros, y otra de gatos. Para que los dataloaders que utilizamos funcionasen de forma correcta, era más fácil dividir las fotos en otras 2 carpetas, una de fotografías de entrenamiento, y otra de fotografías de validación. Dentro de estas 2 carpetas, siguen divididas entre perros y gatos, pero como se ve en el código, las clases son descartadas y no se diferencia entre imágenes de perros y de gatos. La distribución se hizo de tal forma que tenemos un 90% de las fotos para entrenar y el 10% restante para validar.

El funcionamiento de este modelo se basa principalmente en 2 bloques, el codificador (encoder) y decodificador (decoder).
El codificador se encarga de procesar las imágenes de entrada a través de capas convolucionales como si fuese una cnn.
Tras darnos una salida (en este caso son 2 salidas, más adelante se comentan cuales), se pasan por el decodificador, que con esa
salida tratará de generar la misma imagen de entrada. Por ello, el error se calcula utilizando la salida del decoder y las imágenes que entran al encoder.
Si el error es muy bajo, significará que las imágenes generadas son iguales que las de entrada y nuestro autoencoder será capaz de generar fotos de perros y gatos coherentes (o eso creíamos).




### Imports

In [4]:
from __future__ import print_function
from __future__ import division

import os
import shutil
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import models
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import time
from tensorboardX import SummaryWriter
from glob import glob
from util import *
import numpy as np
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

from vae import VAE, ShallowVAE

### Variables globales

In [5]:
""" Como era un .py, en esta parte había diversas instrucciones para leer paramétros de entrada
    Las hemos eliminado al transformarlo en notebook y no ser necesarias
    (pueden verse en el archivo original)
"""
torch.manual_seed(1)
#Automatización de uso de GPU (si es posible) o CPU
if torch.cuda.is_available():
    torch.cuda.manual_seed(1)
    is_cuda = True
else:
    is_cuda = False

#Variables globales
BATCH_SIZE = 128
EPOCH = 20
LOG_INTERVAL=1
path = 'PetImages/'
kwargs = {'num_workers': 3, 'pin_memory': True} if is_cuda else {}

### Carga de datos

In [6]:
#Método para hacer un resize a las imágenes y normalizarlas (para que sirvan de entrada a la red)
simple_transform = transforms.Compose([transforms.Resize((224,224))
                                       ,transforms.ToTensor(), transforms.Normalize([0.48829153, 0.45526633, 0.41688013],[0.25974154, 0.25308523, 0.25552085])])
#Carga de imágenes de entrenamiento y de testing
train = ImageFolder(path+'train',simple_transform)
valid = ImageFolder(path+'valid',simple_transform)
"""Creación de los dataloaders, que se encargarán de cargar los datos en la red
   Crearán lotes de imágenes del batch_size indicado anteriormente
"""
train_data_gen = torch.utils.data.DataLoader(train,shuffle=True,batch_size=BATCH_SIZE,num_workers=kwargs['num_workers'])
valid_data_gen = torch.utils.data.DataLoader(valid,batch_size=BATCH_SIZE,num_workers=kwargs['num_workers'])

#Datos relacionados con los datos (convertidos en diccionarios)
dataset_sizes = {'train':len(train_data_gen.dataset),'valid':len(valid_data_gen.dataset)}
dataloaders = {'train':train_data_gen,'valid':valid_data_gen}


### Modelo

El modelo es un VAE (Autoencoder Variacional) superficial. En el fichero vae.py están ambos
modelos declarados. La diferencia entre ellos es que el superficial tiene un número significativamente inferior de capas interiores para aligerar el peso de la red y el procesado de este.

Este VAE superficial tiene 3 bloques: el encoder, el reparametrizador y el decoder
El encoder reduce el tamaño de las imágenes pasándolas por diferentes capas convolucionales
para extraer las características de las imágenes, hasta convertirlas en un vector.
    
Este encoder consta de 4 sub-bloques que realizan la misma secuencia: aplican un normalizado en lote, aplican una convolución al resultado y una función ReLu, y se pasa al siguiente bloque (el primero no tiene normalizado). Luego aplica capas full-connected y obtiene valores usados posteriormente. Cabe destacar que este encoder tiene 2 salidas (cada una generada por una capa full-connected). Una de las salidas es la media y la otra la desviación típica de los valores de las imágenes, los cuales serán muy útiles para cálculos posteriores.
    
El reparametrizador normaliza las características obtenidas en el encoder, utilizando para ello una distribución normal, y la media y desviación típica obtenidas del propio encoder previamente mencionadas.

El decoder reconstruye las imágenes originales a partir de los datos normalizados obtenidos del reparametrizador. Para ello, aplica dos capas full-connected con su respectiva función ReLu, y luego aplica 4 veces una deconvolución con su respectiva función (ReLu en las 3 primeras y Sigmoide en la última).

In [7]:
#Se declara un modelo que soporta imágenes de 224x224 píxeles RGB (3 canales)
model = ShallowVAE(latent_variable_size=500, nc=3, ngf=224, ndf=224, is_cuda=is_cuda)

#model = VAE(BasicBlock, [2, 2, 2, 2], latent_variable_size=500, nc=3, ngf=224, ndf=224, is_cuda=is_cuda)

if is_cuda:
    model.cuda()

"""Se utiliza el Error Cuadrático Medio (Mean Square Error o MSE) para calcular el error de
   la red (diferencia entre imagen de entrada e imagen de salida)
"""
reconstruction_function = nn.MSELoss()
reconstruction_function.size_average = False

optimizer = optim.Adam(model.parameters(), lr=1e-4)

### Funcion de perdida

Aunque se mencione que la función de error que se utiliza es un MSE, realmente no es lo único que se utiliza. Las salidas del encoder (media y desviación típica) son muy útiles para el cálculo del error, ya que se aplica una fórmula denominada Divergencia de KL (o simplemente KLD). 
El MSE calcula el error pero comparando los valores de cada tensor. No es una medida incorrecta, pero quizás para fotos no es la más precisa porque también deberíamos de tener en cuenta la distribución de la imagen, y es ahí donde entra el KLD, que para su cálculo utiliza las salidas del encoder, que al fin y al cabo son distribuciones probabilísticas. 

In [9]:
#Método para la función de pérdida de la red
def loss_function(recon_x, x, mu, logvar):
    #Se calcula el MSE
    MSE = reconstruction_function(recon_x, x)

    # https://arxiv.org/abs/1312.6114 (Appendix B)
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    
    """ El KLD es una medida sobre la divergencia entre dos distribuciones probabilisticas
    Al usarse la media y la desviación típica para calcular los parámetros, es necesario 
    aplicar el KLD al MSE para reconducir este error (el MSE sólo calcula la diferencia
    entre 2 imágenes, no sus distribuciones)
    """
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)

    return MSE + KLD


### Entrenamiento

In [10]:
def train(epoch):

    #Activa diferentes banderas para que la red pueda entrenar
    model.train()
    train_loss = 0
    batch_idx = 1
    #El dataloader carga lotes para meterlos en la red
    for data in dataloaders['train']:
        #Obtenemos la entrada de la red. El segundo parámetro (la etiqueta) es ignorado
        inputs, _ = data

        #Se envuelven los datos en una Variable (un Tensor al que se le aplica gradiente)
        if torch.cuda.is_available():
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)
        
        #Se resetea el optimizador (para no arrastrar error)
        optimizer.zero_grad()
        #Se pasan los datos por el modelo y se obtienen imágenes
        recon_batch, mu, logvar = model(inputs)
        #print(inputs.data.size())
        
        #Se desnormalizan las imágenes de entrada para que pueda calcularse el error real
        inputs.data = unnormalize(inputs.data,[0.48829153, 0.45526633, 0.41688013],[0.25974154, 0.25308523, 0.25552085])

        #print("input max/min"+str(inputs.max())+"  "+str(inputs.min()))
        #print("recon input max/min"+str(recon_batch.max())+"  "+str(recon_batch.min()))
        
        #Se calcula el error producido por la red y se propaga hacia atrás
        loss = loss_function(recon_batch, inputs, mu, logvar)
        loss.backward()
        train_loss += loss.data
        optimizer.step()

        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(inputs), (len(dataloaders['train'])*128),
                100. * batch_idx / len(dataloaders['train']),
                loss.data / len(inputs)))
        batch_idx+=1

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / (len(dataloaders['train'])*BATCH_SIZE)))
    return train_loss / (len(dataloaders['train'])*BATCH_SIZE)

### Test

In [11]:
def test(epoch):
    #Activa diferentes banderas para que la red pueda evaluarse sin modificarse
    model.eval()
    test_loss = 0
    #El dataloader carga lotes para meterlos en la red
    for data in dataloaders['valid']:
        #Obtenemos la entrada de la red. El segundo parámetro (la etiqueta) es ignorado
        inputs, _ = data

        #Se envuelven los datos en una Variable 
        if torch.cuda.is_available():
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)
        #Se pasan los datos por el modelo y se obtienen imágenes
        recon_batch, mu, logvar = model(inputs)
        #Se desnormalizan las imágenes de entrada para que pueda calcularse el error real
        inputs.data = unnormalize(inputs.data,[0.48829153, 0.45526633, 0.41688013],[0.25974154, 0.25308523, 0.25552085])
        test_loss += loss_function(recon_batch, inputs, mu, logvar).data
        
        #Se guardan las imágenes de entrada y las imágenes de salida (ver último apartado)
        if((epoch+1)%10==0):
            torchvision.utils.save_image(inputs.data, './imgs/Epoch_{}_data.jpg'.format(epoch), nrow=8, padding=2)
            torchvision.utils.save_image(recon_batch.data, './imgs/Epoch_{}_recon.jpg'.format(epoch), nrow=8, padding=2)

    test_loss /= (len(dataloaders['valid'])*128)
    print('====> Test set loss: {:.4f}'.format(test_loss))
    return test_loss



In [12]:
#Crea un fichero "log" para almacenar el error de cada época
writer = SummaryWriter('runs/exp-1')
since = time.time()
#El modelo entrena las épocas indicadas, y por cada época de entrenamiento, hace una validación a la red
for epoch in range(EPOCH):
    train_loss = train(epoch)
    test_loss = test(epoch)
    writer.add_scalar('train_loss', train_loss, epoch)
    writer.add_scalar('test_loss',test_loss, epoch)
    #Guarda los parámetros de la red en cada iteración (para un posible comeback si la red se desajusta)
    torch.save(model.state_dict(), './models/Epoch_{}_Train_loss_{:.4f}_Test_loss_{:.4f}.pth'.format(epoch, train_loss, test_loss))
time_elapsed = time.time() - since    
print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

Train Epoch: 0 [128/22528 (1%)]	Loss: 749.054626
Train Epoch: 0 [256/22528 (1%)]	Loss: 437.616852
Train Epoch: 0 [384/22528 (2%)]	Loss: 270.843384
Train Epoch: 0 [512/22528 (2%)]	Loss: 169.746857
Train Epoch: 0 [640/22528 (3%)]	Loss: 110.126137
Train Epoch: 0 [768/22528 (3%)]	Loss: 73.289856
Train Epoch: 0 [896/22528 (4%)]	Loss: 47.206734
Train Epoch: 0 [1024/22528 (5%)]	Loss: 31.948927
Train Epoch: 0 [1152/22528 (5%)]	Loss: 21.818117
Train Epoch: 0 [1280/22528 (6%)]	Loss: 14.836316
Train Epoch: 0 [1408/22528 (6%)]	Loss: 9.256569
Train Epoch: 0 [1536/22528 (7%)]	Loss: 7.082623
Train Epoch: 0 [1664/22528 (7%)]	Loss: 4.929274
Train Epoch: 0 [1792/22528 (8%)]	Loss: 3.830462
Train Epoch: 0 [1920/22528 (9%)]	Loss: 2.794017
Train Epoch: 0 [2048/22528 (9%)]	Loss: 2.045649
Train Epoch: 0 [2176/22528 (10%)]	Loss: 1.489249
Train Epoch: 0 [2304/22528 (10%)]	Loss: 1.241693
Train Epoch: 0 [2432/22528 (11%)]	Loss: 1.037365
Train Epoch: 0 [2560/22528 (11%)]	Loss: 0.727125
Train Epoch: 0 [2688/22528 (

### Problema observados

La red es capaz de ajustar de manera sobresaliente en relativamente poco tiempo, sin embargo, hay algún problema a la hora de visualizar las imágenes resultantes. Cuando las imágenes de entrada se guardan, pueden visualizarse en la carpeta correspondiente con buena calidad, pero las imágenes generadas aparecen como imágenes en gris (independientemente de si se reentrena desde 0), por lo que suponemos que es algo relacionado al guardado de estas.

Con respecto a la futura generación de imágenes cuando la red está ya entrenada, se necesitaría investigar qué es necesario aportar como entrada a la red para que la genere, pero ya que no es posible visualizar la salida correctamente, no podría comprobarse que funciona bien para estos casos.

Esto podría haberse solucionado quizá con suficiente tiempo para investigar y experimentar, pero debido a más proyectos y exámenes, no nos ha sido posible :"(  (lo sentimos Paco, nos hubiera gustado de verdad poder sacar más tiempo)